In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy.optimize import minimize, fmin_tnc
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('optdigits.tra',header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0,1,6,15,12,1,0,0,0,7,...,0,0,0,6,14,7,1,0,0,0
1,0,0,10,16,6,0,0,0,0,7,...,0,0,0,10,16,15,3,0,0,0
2,0,0,8,15,16,13,0,0,0,1,...,0,0,0,9,14,0,0,0,0,7
3,0,0,0,3,11,16,0,0,0,0,...,0,0,0,0,1,15,2,0,0,4
4,0,0,5,14,4,0,0,0,0,0,...,0,0,0,4,12,14,7,0,0,6


In [4]:
df_feat = df.iloc[:,:64]

In [5]:
df_out = df.iloc[:,64]

In [6]:
X_train = np.c_[np.ones((df_feat.shape[0], 1)), df_feat]

In [7]:
X_train

array([[ 1.,  0.,  1., ...,  1.,  0.,  0.],
       [ 1.,  0.,  0., ...,  3.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 1.,  0.,  0., ...,  9.,  0.,  0.],
       [ 1.,  0.,  0., ..., 16.,  5.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [8]:
y = np.array(df_out)[:, np.newaxis]

In [9]:
X_train.shape

(3823, 65)

In [10]:
y.shape

(3823, 1)

In [11]:
def y_change(y, cl):
    y_pr=[]
    for i in range(0, len(y)):
        if y[i] == cl:
            y_pr.append(1)
        else:
            y_pr.append(0)
    return y_pr

In [12]:
def hypothesis(X, theta):
    z = np.dot(X, theta)
    return 1/(1+np.exp(-(z)))

In [13]:
def cost_function(theta, X, y):
    m = X.shape[0]
    y1 = hypothesis(X, theta)
    return -(1/len(X)) * np.sum(y*np.log(y1) + (1-y)*np.log(1-y1))

In [14]:
# update theta
def gradient(theta, X, y):
    m = X.shape[0]
    y1 = hypothesis(X, theta)
    return (1/m) * np.dot(X.T, y1 - y)

In [31]:
theta = np.zeros((X_train.shape[1], 1))

In [19]:
def fit(X, y, theta):
    opt_weigths = fmin_tnc(func=cost_function, x0=theta,
                          fprime=gradient, args=(X, y.flatten()))
    return opt_weigths[0]

In [20]:
def find_param(X, y, theta):
    y_uniq = list(set(y.flatten()))
    theta_list = []
    for i in y_uniq:
        y_tr = pd.Series(y_change(y, i))
        y_tr = y_tr[:, np.newaxis]
        theta1 = fit(X, y, theta)
        theta_list.append(theta1)
    return theta_list

In [32]:
theta_list = find_param(X_train, y, theta)

In [33]:
def predict(theta_list, X, y):
    y_uniq = list(set(y.flatten()))
    y_hat = [0]*len(y)
    for i in range(0, len(y_uniq)):
        y_tr = y_change(y, y_uniq[i])
        y1 = hypothesis(X, theta_list[i])
        for k in range(0, len(y)):
            if y_tr[k] == 1 and y1[k] >= 0.5:
                y_hat[k] = y_uniq[i]
    return y_hat

In [34]:
y_hat = predict(theta_list, X_train, y)

In [35]:
accuracy=0
for i in range(0, len(y)):
    if y_hat[i] == y.flatten()[i]:
        accuracy += 1
print(accuracy/len(df_feat)*100)

100.0


In [44]:
df_test = pd.read_csv('optdigits.tes',header=None)

In [45]:
df_test_feat = df_test.iloc[:,:64]

In [46]:
df_test_out = df_test.iloc[:,64]

In [50]:
X_test = np.c_[np.ones((df_test_feat.shape[0], 1)), df_test_feat]

In [52]:
y_test = np.array(df_test_out)[:, np.newaxis]

In [54]:
y_hat_test = predict(theta_list,X_test,y_test)

In [55]:
accuracy_test=0
for i in range(0, len(y_test)):
    if y_hat_test[i] == y_test.flatten()[i]:
        accuracy_test += 1
print(accuracy_test/len(df_test_feat)*100)

100.0


In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report

In [57]:
model = LogisticRegression()

In [58]:
model.fit(X_train,y)

LogisticRegression()

In [59]:
y_pred = model.predict(X_test)

In [61]:
confusion_matrix(y_test,y_pred)

array([[174,   0,   0,   0,   0,   4,   0,   0,   0,   0],
       [  0, 176,   0,   0,   1,   0,   0,   0,   4,   1],
       [  0,   4, 172,   0,   0,   0,   1,   0,   0,   0],
       [  1,   0,   4, 168,   0,   3,   0,   1,   1,   5],
       [  0,   2,   0,   0, 172,   0,   0,   1,   2,   4],
       [  0,   0,   1,   1,   0, 175,   0,   1,   0,   4],
       [  2,   0,   0,   0,   0,   0, 178,   0,   1,   0],
       [  0,   0,   0,   0,   1,   8,   0, 161,   1,   8],
       [  0,   7,   0,   1,   1,   3,   1,   0, 155,   6],
       [  1,   0,   0,   0,   2,   2,   0,   0,   3, 172]])

In [63]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       178
           1       0.93      0.97      0.95       182
           2       0.97      0.97      0.97       177
           3       0.99      0.92      0.95       183
           4       0.97      0.95      0.96       181
           5       0.90      0.96      0.93       182
           6       0.99      0.98      0.99       181
           7       0.98      0.90      0.94       179
           8       0.93      0.89      0.91       174
           9       0.86      0.96      0.91       180

    accuracy                           0.95      1797
   macro avg       0.95      0.95      0.95      1797
weighted avg       0.95      0.95      0.95      1797



In [64]:
from sklearn.linear_model import Ridge

In [65]:
md = Ridge()